In [1]:
from disambiguation.core import Database
from disambiguation import config
from cPickle import load
import pandas as pd
from glob import glob
%load_ext autoreload
import re
import json
%autoreload 2

individual_contributions_v2
newyork_combined_v2
usa_combined_v2
identities_v3
identities_adjacency_v3
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Export full tables, with additional identity column to file. 

In [2]:
idm = Database.IdentityManager('USA')

Table 'identities_v3' exists.
Table 'identities_adjacency_v3' exists.


In [9]:
for state in ['districtofcolumbia','virginia','maryland']:
# for state in ['virginia']:
    print state
    exporter = Database.FecExporter(state)
    exporter.retrieve()
    exporter.set_idm(idm)
    exporter.export_csv()

districtofcolumbia
select NAME,ZIP_CODE,CONTRIBUTOR_STREET_1,CITY,STATE,EMPLOYER,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,CMTE_ID,ENTITY_TP,id from districtofcolumbia_combined;
virginia
select NAME,ZIP_CODE,CONTRIBUTOR_STREET_1,CITY,STATE,EMPLOYER,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,CMTE_ID,ENTITY_TP,id from virginia_combined;
maryland
select NAME,ZIP_CODE,CONTRIBUTOR_STREET_1,CITY,STATE,EMPLOYER,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,CMTE_ID,ENTITY_TP,id from maryland_combined;


In [5]:
exporter.export_csv('delaware.csv')


Table 'identities' exists.
Table 'identities_adjacency' exists.
select id,identity from identities;


#Jierui 

In [13]:
coders = ['guest1','guest2','guest3','guest4','guest5','guest6','guest7','guest8','guest9']
data = {coder:{} for coder in coders}
for coder in coders:
    files = '/nfs/home/navid/data/Jierui/board-data-top500-400left/{}*.json'.format(coder)
    list_files = glob(files)
#     print list_files
#     continue
    for filename in list_files:
#         print filename
        file_id = re.findall(r'-([0-9]+)\.',filename)[0]
        f = open(filename)
        tmp = json.load(f) 
        data[coder][file_id] = tmp


In [14]:
counter = 0
# list_rids = []
set_rids = set()
for coder in coders:
    with open('SP500_400left_{}_results.csv'.format(coder),'w') as f:
        for pageid, pagedata in sorted(data[coder].items(), key = lambda x:int(x[0])):
            list_yesses = [key for key,value in pagedata['data'].iteritems() if value[2] == 1]
            set_rids.update( {int(x) for x in list_yesses})
            f.write(pageid + " " + ' '.join(list_yesses) + "\n")

            counter += 1
    #         if counter > 20: break
    # print list_rids

## Export all the "yes" records

In [16]:
exporter = Database.FecRetrieverByID('usa_combined_v1')
exporter.retrieve(list(set_rids))
exporter.set_idm(idm)
exporter.export_csv('./all_records_sp500_400left')

inserting...
done.
Done in 8.299324 seconds
SELECT NAME,ZIP_CODE,CONTRIBUTOR_STREET_1,CITY,STATE,EMPLOYER,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,CMTE_ID,ENTITY_TP,id FROM usa_combined_v1 JOIN tmp_QX9QCB0k00NjCchOht1i USING (id) ;
Done in 31.283689 seconds
dict_id_2_identity not loaded. Loading now...
select id,identity from identities_v2;


/nfs/home/navid/src/FEC-1/src/disambiguation/core/Database.py:52: Warning: Unknown table 'tmp_QX9QCB0k00NjCchOht1i'
  cur.execute(query)


# Carolina

In [4]:
coders = ['carolina']
data = {coder:{} for coder in coders}
for coder in coders:
    files = '/nfs/home/navid/data/Carolina/{}*.json'.format(coder)
    list_files = glob(files)
#     print list_files
    for filename in list_files:
#         print filename
        file_id = re.findall(r'-([0-9]+)\.',filename)[0]
        f = open(filename)
        tmp = json.load(f) 
        data[coder][file_id] = tmp

In [19]:
counter = 0
# list_rids = []
set_rids = set()
dict_rids = {}
for coder in coders:
    with open('carolina_{}_results.csv'.format(coder),'w') as f:
        for pageid, pagedata in sorted(data[coder].items(), key = lambda x:int(x[0])):
            list_yesses = [key for key,value in pagedata['data']['final'].iteritems() if value[1] == 1]
#             print list_yesses
            set_rids.update( {int(x) for x in list_yesses})
            dict_rids[int(pageid)] = [int(x) for x in list_yesses]
            f.write(pageid + " " + ' '.join(list_yesses) + "\n")

            counter += 1
    #         if counter > 20: break
    # print list_rids

In [14]:
print dict_rids

{1: [u' 10230295 ', u' 10314752 ', u' 12616865 ', u' 13674235 ', u' 377534 '], 2: [u' 3919294 ', u' 5637966 ', u' 7269837 ', u' 4850178 ', u' 6661654 ', u' 10994088 ', u' 5148397 ', u' 5310995 ', u' 7477602 '], 3: [u' 9964145 ', u' 10302768 ', u' 10818233 ', u' 10436561 ', u' 9809241 ', u' 10531403 ', u' 9679323 ', u' 5858052 ', u' 10001664 ', u' 5230808 ', u' 8714669 ', u' 7102798 ', u' 9280841 ', u' 11898883 ', u' 9147151 ', u' 6471130 ', u' 12533563 ', u' 10102276 '], 4: [u' 11122452 ', u' 10869025 ', u' 13263222 ', u' 6614784 ', u' 4382352 ', u' 13911049 ', u' 13276677 ', u' 12385388 ', u' 12242215 ', u' 6418779 ', u' 5318207 ', u' 10356783 ', u' 6900394 ', u' 10998432 ', u' 13969676 ', u' 4627535 ', u' 14316589 ', u' 11485904 ', u' 11626097 ', u' 12911978 ', u' 12123981 ', u' 11189543 ', u' 6735851 ', u' 6206326 ', u' 8941335 ', u' 7243690 ', u' 5020072 ', u' 9943683 ', u' 12470558 ', u' 14600903 ', u' 5789993 ', u' 6042760 ', u' 6376617 ', u' 12742820 ', u' 5956711 '], 5: [u' 215

## Export all "yes" records

In [20]:
exporter = Database.FecRetrieverByID('usa_combined')
exporter.set_idm(idm)

for pageid, list_yesses in dict_rids.items():
    print pageid
    exporter.retrieve(list_yesses)
    filename = "carolina_records_page_{}".format(pageid)
    exporter.export_csv(filename)

1
inserting...
done.
Done in 0.001655 seconds
SELECT NAME,ZIP_CODE,CONTRIBUTOR_STREET_1,CITY,STATE,EMPLOYER,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,CMTE_ID,ENTITY_TP,id FROM usa_combined JOIN tmp_rIBsGSlfQ71XV4MkxE3o USING (id) ;
Done in 16.159047 seconds
2

/nfs/home/navid/src/FEC-1/src/disambiguation/core/Database.py:51: Warning: Unknown table 'tmp_rIBsGSlfQ71XV4MkxE3o'
  cur.execute(query)
/nfs/home/navid/src/FEC-1/src/disambiguation/core/Database.py:51: Warning: Unknown table 'tmp_TavkJgTAQoYoNGLdMqCp'
  cur.execute(query)



inserting...
done.
Done in 0.001603 seconds
SELECT NAME,ZIP_CODE,CONTRIBUTOR_STREET_1,CITY,STATE,EMPLOYER,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,CMTE_ID,ENTITY_TP,id FROM usa_combined JOIN tmp_TavkJgTAQoYoNGLdMqCp USING (id) ;
Done in 24.502892 seconds
3
inserting...
done.
Done in 0.002647 seconds
SELECT NAME,ZIP_CODE,CONTRIBUTOR_STREET_1,CITY,STATE,EMPLOYER,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,CMTE_ID,ENTITY_TP,id FROM usa_combined JOIN tmp_hGBbWXEfCcm8bBe0wDPt USING (id) ;
Done in 23.350559 seconds
4

/nfs/home/navid/src/FEC-1/src/disambiguation/core/Database.py:51: Warning: Unknown table 'tmp_hGBbWXEfCcm8bBe0wDPt'
  cur.execute(query)
/nfs/home/navid/src/FEC-1/src/disambiguation/core/Database.py:51: Warning: Unknown table 'tmp_8VmbWqRDWcI5FI5mEXrO'
  cur.execute(query)



inserting...
done.
Done in 0.003184 seconds
SELECT NAME,ZIP_CODE,CONTRIBUTOR_STREET_1,CITY,STATE,EMPLOYER,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,CMTE_ID,ENTITY_TP,id FROM usa_combined JOIN tmp_8VmbWqRDWcI5FI5mEXrO USING (id) ;
Done in 22.369494 seconds
5
inserting...
done.
Done in 0.018382 seconds
SELECT NAME,ZIP_CODE,CONTRIBUTOR_STREET_1,CITY,STATE,EMPLOYER,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,CMTE_ID,ENTITY_TP,id FROM usa_combined JOIN tmp_Ez9Oj6rmx439j7pTUwWu USING (id) ;
Done in 14.457346 seconds
6

...


inserting...
done.
Done in 0.000744 seconds
SELECT NAME,ZIP_CODE,CONTRIBUTOR_STREET_1,CITY,STATE,EMPLOYER,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,CMTE_ID,ENTITY_TP,id FROM usa_combined JOIN tmp_nWlB9ZdgFDhJK4SN6l9u USING (id) ;
Done in 13.979682 seconds
85
inserting...
done.
Done in 0.001799 seconds
SELECT NAME,ZIP_CODE,CONTRIBUTOR_STREET_1,CITY,STATE,EMPLOYER,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,CMTE_ID,ENTITY_TP,id FROM usa_combined JOIN tmp_X2i6hhnPMLINffggqpbD USING (id) ;
Done in 22.167052 seconds
86

/nfs/home/navid/src/FEC-1/src/disambiguation/core/Database.py:51: Warning: Unknown table 'tmp_X2i6hhnPMLINffggqpbD'
  cur.execute(query)
/nfs/home/navid/src/FEC-1/src/disambiguation/core/Database.py:51: Warning: Unknown table 'tmp_rd2HrrIzNmC0wELPnMVn'
  cur.execute(query)



inserting...
done.
Done in 0.000789 seconds
SELECT NAME,ZIP_CODE,CONTRIBUTOR_STREET_1,CITY,STATE,EMPLOYER,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,CMTE_ID,ENTITY_TP,id FROM usa_combined JOIN tmp_rd2HrrIzNmC0wELPnMVn USING (id) ;
Done in 13.953395 seconds
87
inserting...
done.
Done in 0.004701 seconds
SELECT NAME,ZIP_CODE,CONTRIBUTOR_STREET_1,CITY,STATE,EMPLOYER,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,CMTE_ID,ENTITY_TP,id FROM usa_combined JOIN tmp_NiEMiUR022eXmDW11sbz USING (id) ;
Done in 28.924230 seconds
88

/nfs/home/navid/src/FEC-1/src/disambiguation/core/Database.py:51: Warning: Unknown table 'tmp_NiEMiUR022eXmDW11sbz'
  cur.execute(query)
/nfs/home/navid/src/FEC-1/src/disambiguation/core/Database.py:51: Warning: Unknown table 'tmp_x1MrUbMVt7zQAs7JT32s'
  cur.execute(query)



inserting...
done.
Done in 0.001680 seconds
SELECT NAME,ZIP_CODE,CONTRIBUTOR_STREET_1,CITY,STATE,EMPLOYER,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,CMTE_ID,ENTITY_TP,id FROM usa_combined JOIN tmp_x1MrUbMVt7zQAs7JT32s USING (id) ;
Done in 32.050410 seconds


In [1]:
idm

NameError: name 'idm' is not defined

# Test identity adjacency


In [26]:
# idm.fetch_dict_identity_2_id()
# idm.fetch_dict_id_2_identity
idm.load_dict_identity_2_identities()

In [16]:
idm.fetch_dict_identity_adjacency()


SELECT identity1,identity2, verdict from identities_adjacency_v3;


In [18]:
idm.dict_identity_adjacency['OK-3394']

KeyError: 'OK-3394'

In [29]:
idm.dict_identity_2_identities['OK-3399']

KeyError: 'OK-3399'

In [25]:
idm.get_linked_identities('OK-3323')

[u'OK-3394', 'OK-3323']

In [32]:
import cPickle
with open(config.tokendata_file_template % ( 'newyork', 'Tokenizer')) as f:
    a = cPickle.load(f)
    

In [34]:
a.token_identifiers

{'CONTRIBUTOR_STREET_1': [5],
 'CONTRIBUTOR_ZIP': [4],
 'EMPLOYER': [67],
 'FIRST_NAME': [2],
 'LAST_NAME': [1],
 'MIDDLE_NAME': [3],
 'NAME': [1, 2, 3],
 'N_employer': [67],
 'N_first_name': [2],
 'N_full_name': [123],
 'N_last_name': [1],
 'N_middle_name': [3],
 'N_occupation': [67],
 'OCCUPATION': [67],
 'ZIP_CODE': [4]}